In [1]:
import pandas as pd
import numpy as np
import mysql.connector
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score

In [2]:
random_df = pd.DataFrame()
print(random_df)

Empty DataFrame
Columns: []
Index: []


In [3]:
connection = mysql.connector.connect(
            host = "localhost",
            port = 3306,
            user = 'root',
            database = "expandinghorizons",
            password = "dB79@dG2024!"
)
    
cursor = connection.cursor()

query = ("SELECT distinct child_id FROM child ")
cursor.execute(query)

child_ids = cursor.fetchall()

data_list = []
for row in child_ids:
    data = {}
    data["id"] = row[0]
    child1_child_query = ("SELECT child_id_1, child_id_2, driving_distance, driving_time "
                          "FROM locationdataalgo2 "
                          f"WHERE child_id_1 = '{row[0]}' AND ei_id_1 IS NULL AND ei_id_2 IS NULL")
    cursor.execute(child1_child_query)
    child1_child = cursor.fetchall()
    
    for row2 in child1_child:
        # Add columns with prefixes
        data[f'{row2[1]}_distance'] = row2[2]
        data[f'{row2[1]}_time'] = row2[3]
        
    child_child2_query = ("SELECT child_id_1, child_id_2, driving_distance, driving_time "
                          "FROM locationdataalgo2 "
                          f"WHERE child_id_2 = '{row[0]}' AND ei_id_1 IS NULL AND ei_id_2 IS NULL")
    cursor.execute(child_child2_query)
    child_child2 = cursor.fetchall()
    
    for row3 in child_child2:
        data[f'{row3[0]}_distance'] = row3[2]
        data[f'{row3[0]}_time'] = row3[3]
    
    child_ei_query = ("SELECT child_id_1, ei_id_1, driving_distance, driving_time "
                      "FROM locationdataalgo2 "
                      f"WHERE child_id_1 = '{row[0]}' AND child_id_2 IS NULL AND ei_id_2 IS NULL")
    cursor.execute(child_ei_query)
    child_ei = cursor.fetchall()
    
    for row4 in child_ei:
        data[f'{row4[1]}_distance'] = row4[2]
        data[f'{row4[1]}_time'] = row4[3]
        
    assigned_query = ("SELECT ei_id_1 "
                      "FROM locationdataalgo2 "
                      f"WHERE child_id_1 = '{row[0]}' and assigned = 1")
    cursor.execute(assigned_query)
    assigned = cursor.fetchall()
    
    data["assigned_ei"] = assigned[0][0]
        
    data_list.append(data)
    
cursor.close()
connection.close()
        
random_df = pd.DataFrame(data_list)

# Display the DataFrame
print(random_df)

                           id  65d8d732e09e67e025e8cdb6_distance  \
0    65d76430e09e67e025e8cd43                            4.95918   
1    65d764dde09e67e025e8cd44                            5.41753   
2    65d766f6e09e67e025e8cd45                           42.51830   
3    65d76799e09e67e025e8cd46                           23.30160   
4    65d76829e09e67e025e8cd47                           33.43960   
..                        ...                                ...   
191  65e6038938f826f29267202f                           24.44470   
192  65e603f238f826f292672030                           32.58570   
193  65e6044938f826f292672031                           23.77980   
194  65e6049b38f826f292672032                           23.28850   
195  65ef076606e0a866af56d202                           39.28150   

     65d8d732e09e67e025e8cdb6_time  65d8d7c4e09e67e025e8cdb7_distance  \
0                          10.3572                            4.95918   
1                          13.4520   

In [4]:
print(random_df.isnull().sum())

id                                   0
65d8d732e09e67e025e8cdb6_distance    1
65d8d732e09e67e025e8cdb6_time        1
65d8d7c4e09e67e025e8cdb7_distance    1
65d8d7c4e09e67e025e8cdb7_time        1
                                    ..
65d75e28e09e67e025e8cd3c_distance    0
65d75e28e09e67e025e8cd3c_time        0
assigned_ei                          0
65d76430e09e67e025e8cd43_distance    1
65d76430e09e67e025e8cd43_time        1
Length: 420, dtype: int64


In [5]:
random_df = random_df.fillna(0)
print(random_df)

                           id  65d8d732e09e67e025e8cdb6_distance  \
0    65d76430e09e67e025e8cd43                            4.95918   
1    65d764dde09e67e025e8cd44                            5.41753   
2    65d766f6e09e67e025e8cd45                           42.51830   
3    65d76799e09e67e025e8cd46                           23.30160   
4    65d76829e09e67e025e8cd47                           33.43960   
..                        ...                                ...   
191  65e6038938f826f29267202f                           24.44470   
192  65e603f238f826f292672030                           32.58570   
193  65e6044938f826f292672031                           23.77980   
194  65e6049b38f826f292672032                           23.28850   
195  65ef076606e0a866af56d202                           39.28150   

     65d8d732e09e67e025e8cdb6_time  65d8d7c4e09e67e025e8cdb7_distance  \
0                          10.3572                            4.95918   
1                          13.4520   

In [6]:
popped_column = random_df.pop("assigned_ei")
random_df["assigned_ei"] = popped_column
print(random_df)

                           id  65d8d732e09e67e025e8cdb6_distance  \
0    65d76430e09e67e025e8cd43                            4.95918   
1    65d764dde09e67e025e8cd44                            5.41753   
2    65d766f6e09e67e025e8cd45                           42.51830   
3    65d76799e09e67e025e8cd46                           23.30160   
4    65d76829e09e67e025e8cd47                           33.43960   
..                        ...                                ...   
191  65e6038938f826f29267202f                           24.44470   
192  65e603f238f826f292672030                           32.58570   
193  65e6044938f826f292672031                           23.77980   
194  65e6049b38f826f292672032                           23.28850   
195  65ef076606e0a866af56d202                           39.28150   

     65d8d732e09e67e025e8cdb6_time  65d8d7c4e09e67e025e8cdb7_distance  \
0                          10.3572                            4.95918   
1                          13.4520   

In [7]:
random_df = random_df.drop(columns=["id"])
print(random_df)

     65d8d732e09e67e025e8cdb6_distance  65d8d732e09e67e025e8cdb6_time  \
0                              4.95918                        10.3572   
1                              5.41753                        13.4520   
2                             42.51830                        57.0214   
3                             23.30160                        34.1519   
4                             33.43960                        58.7406   
..                                 ...                            ...   
191                           24.44470                        38.0402   
192                           32.58570                        38.5214   
193                           23.77980                        38.4946   
194                           23.28850                        36.5575   
195                           39.28150                        69.2705   

     65d8d7c4e09e67e025e8cdb7_distance  65d8d7c4e09e67e025e8cdb7_time  \
0                              4.95918            

In [8]:
Y = random_df["assigned_ei"].values
X = random_df.drop(["assigned_ei"], axis=1)
print(X)
print(Y)

     65d8d732e09e67e025e8cdb6_distance  65d8d732e09e67e025e8cdb6_time  \
0                              4.95918                        10.3572   
1                              5.41753                        13.4520   
2                             42.51830                        57.0214   
3                             23.30160                        34.1519   
4                             33.43960                        58.7406   
..                                 ...                            ...   
191                           24.44470                        38.0402   
192                           32.58570                        38.5214   
193                           23.77980                        38.4946   
194                           23.28850                        36.5575   
195                           39.28150                        69.2705   

     65d8d7c4e09e67e025e8cdb7_distance  65d8d7c4e09e67e025e8cdb7_time  \
0                              4.95918            

In [9]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
print(X_scaled)

[[0.06618718 0.11946324 0.06618718 ... 0.31560908 0.         0.        ]
 [0.0723045  0.15515965 0.0723045  ... 0.35211189 0.13221158 0.23679365]
 [0.56746603 0.65770296 0.56746603 ... 0.58246991 0.49834108 0.58760349]
 ...
 [0.3173746  0.44400896 0.3173746  ... 0.37193465 0.37012742 0.46847261]
 [0.31081752 0.42166583 0.31081752 ... 0.36222213 0.36368961 0.44656155]
 [0.52426642 0.79898798 0.52426642 ... 0.97082937 0.50940316 0.77429987]]


In [10]:
silhouette_avg = silhouette_score(X_scaled, Y)
print(silhouette_avg)

0.19275305008926
